In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import os
os.chdir("D:/Training/Academy/ML(Python)/Cases/Glass_Identification")

In [4]:
glass = pd.read_csv("Glass.csv")
X, y = glass.drop('Type', axis=1), glass['Type']
le = LabelEncoder()
y = le.fit_transform(y)
lr = LogisticRegression()
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=25)

In [ ]:
params = {'solver':['lbfgs','newton-cg','newton-cholesky','sag','saga'],
          'C':np.linspace(0.001, 15, 20),'penalty':['l2', None]  }
gcv = GridSearchCV(lr, param_grid=params, scoring='f1_macro', cv=kfold)
gcv.fit(X, y)

In [7]:
print(gcv.best_params_)
print(gcv.best_score_)

{'C': 0.001, 'penalty': None, 'solver': 'newton-cholesky'}
0.595134077823238


In [8]:
df_cv = pd.DataFrame( gcv.cv_results_ )
df_cv

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.022122,0.007800,0.005115,0.003258,0.001,l2,lbfgs,"{'C': 0.001, 'penalty': 'l2', 'solver': 'lbfgs'}",0.090395,0.086207,0.086207,0.086207,0.087719,0.087347,0.001633,196
1,0.014228,0.005019,0.000000,0.000000,0.001,l2,newton-cg,"{'C': 0.001, 'penalty': 'l2', 'solver': 'newto...",0.090395,0.086207,0.086207,0.086207,0.087719,0.087347,0.001633,196
2,0.004478,0.007439,0.005166,0.005675,0.001,l2,newton-cholesky,"{'C': 0.001, 'penalty': 'l2', 'solver': 'newto...",0.090395,0.086207,0.086207,0.086207,0.087719,0.087347,0.001633,196
3,0.003379,0.005216,0.003467,0.004919,0.001,l2,sag,"{'C': 0.001, 'penalty': 'l2', 'solver': 'sag'}",0.090395,0.086207,0.086207,0.086207,0.087719,0.087347,0.001633,196
4,0.004969,0.006377,0.000801,0.001603,0.001,l2,saga,"{'C': 0.001, 'penalty': 'l2', 'solver': 'saga'}",0.090395,0.086207,0.086207,0.086207,0.087719,0.087347,0.001633,196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.021045,0.006500,0.005009,0.006067,15.000,None,lbfgs,"{'C': 15.0, 'penalty': None, 'solver': 'lbfgs'}",0.371946,0.357341,0.452814,0.471635,0.578348,0.446417,0.079452,84
196,0.467049,0.125936,0.000000,0.000000,15.000,None,newton-cg,"{'C': 15.0, 'penalty': None, 'solver': 'newton...",0.583120,0.497136,0.451515,0.475589,0.644048,0.530282,0.072151,55
197,0.022575,0.008810,0.000401,0.000802,15.000,None,newton-cholesky,"{'C': 15.0, 'penalty': None, 'solver': 'newton...",0.585819,0.583310,0.469078,0.581954,0.755509,0.595134,0.091663,1
198,0.007359,0.007715,0.003691,0.007383,15.000,None,sag,"{'C': 15.0, 'penalty': None, 'solver': 'sag'}",0.293182,0.356759,0.262963,0.306101,0.302551,0.304311,0.030301,129


In [10]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
params = {'max_depth':[None, 3, 4,5,6,7],
          'min_samples_split': [2, 10, 0.025, 0.01 ,0.05, 0.1],
          'min_samples_leaf':[1, 10, 0.025, 0.01 ,0.05, 0.1]  }
dtc = DecisionTreeClassifier(random_state=25)
gcv = GridSearchCV(dtc, param_grid=params, scoring='f1_macro', cv=kfold, verbose=3)
gcv.fit(X, y)
print(gcv.best_params_)
print(gcv.best_score_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV 1/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.567 total time=   0.0s
[CV 2/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.634 total time=   0.0s
[CV 3/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.619 total time=   0.0s
[CV 4/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.467 total time=   0.0s
[CV 5/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.657 total time=   0.0s
[CV 1/5] END max_depth=None, min_samples_leaf=1, min_samples_split=10;, score=0.692 total time=   0.0s
[CV 2/5] END max_depth=None, min_samples_leaf=1, min_samples_split=10;, score=0.634 total time=   0.0s
[CV 3/5] END max_depth=None, min_samples_leaf=1, min_samples_split=10;, score=0.635 total time=   0.0s
[CV 4/5] END max_depth=None, min_samples_leaf=1, min_samples_split=10;, score=0.361 total time=   0.0s
[CV 5/5] END m

In [21]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

w/o Pipeline

In [18]:
svm = SVC(kernel='linear')
params = {'C':np.linspace(0.01, 5, 15), 'decision_function_shape':['ovo','ovr']}
gcv = GridSearchCV(svm, param_grid=params, scoring='f1_macro', cv=kfold, verbose=3)
gcv.fit(X, y)
print(gcv.best_params_)
print(gcv.best_score_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV 1/5] END C=0.01, decision_function_shape=ovo;, score=0.250 total time=   0.0s
[CV 2/5] END C=0.01, decision_function_shape=ovo;, score=0.236 total time=   0.0s
[CV 3/5] END C=0.01, decision_function_shape=ovo;, score=0.216 total time=   0.0s
[CV 4/5] END C=0.01, decision_function_shape=ovo;, score=0.179 total time=   0.0s
[CV 5/5] END C=0.01, decision_function_shape=ovo;, score=0.255 total time=   0.0s
[CV 1/5] END C=0.01, decision_function_shape=ovr;, score=0.250 total time=   0.0s
[CV 2/5] END C=0.01, decision_function_shape=ovr;, score=0.236 total time=   0.0s
[CV 3/5] END C=0.01, decision_function_shape=ovr;, score=0.216 total time=   0.0s
[CV 4/5] END C=0.01, decision_function_shape=ovr;, score=0.179 total time=   0.0s
[CV 5/5] END C=0.01, decision_function_shape=ovr;, score=0.255 total time=   0.0s
[CV 1/5] END C=0.36642857142857144, decision_function_shape=ovo;, score=0.360 total time=   0.0s
[CV 2/5] END C=0.3664

with Pipeline

In [22]:
svm = SVC(kernel='linear')
scaler = StandardScaler()
mm = MinMaxScaler()
pipe = Pipeline([('SCL', scaler), ('SVM', svm)])
params = {'SVM__C':np.linspace(0.01, 5, 15), 'SVM__decision_function_shape':['ovo','ovr'],
          'SCL': [mm, scaler]}
gcv = GridSearchCV(pipe, param_grid=params, scoring='f1_macro', cv=kfold, verbose=3)
gcv.fit(X, y)
print(gcv.best_params_)
print(gcv.best_score_)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV 1/5] END SCL=MinMaxScaler(), SVM__C=0.01, SVM__decision_function_shape=ovo;, score=0.090 total time=   0.0s
[CV 2/5] END SCL=MinMaxScaler(), SVM__C=0.01, SVM__decision_function_shape=ovo;, score=0.086 total time=   0.0s
[CV 3/5] END SCL=MinMaxScaler(), SVM__C=0.01, SVM__decision_function_shape=ovo;, score=0.086 total time=   0.0s
[CV 4/5] END SCL=MinMaxScaler(), SVM__C=0.01, SVM__decision_function_shape=ovo;, score=0.086 total time=   0.0s
[CV 5/5] END SCL=MinMaxScaler(), SVM__C=0.01, SVM__decision_function_shape=ovo;, score=0.088 total time=   0.0s
[CV 1/5] END SCL=MinMaxScaler(), SVM__C=0.01, SVM__decision_function_shape=ovr;, score=0.090 total time=   0.0s
[CV 2/5] END SCL=MinMaxScaler(), SVM__C=0.01, SVM__decision_function_shape=ovr;, score=0.086 total time=   0.0s
[CV 3/5] END SCL=MinMaxScaler(), SVM__C=0.01, SVM__decision_function_shape=ovr;, score=0.086 total time=   0.0s
[CV 4/5] END SCL=MinMaxScaler(), SVM__C=0.